<a href="https://colab.research.google.com/github/alex-jk/datakit-smallholder-farmers-fall-2025/blob/main/alex_j_farmers_data_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Clone the repo**

In [ ]:
import os
import pandas as pd
from google.colab import drive

repo_dir = "datakit-smallholder-farmers-fall-2025"

if not os.path.isdir(repo_dir):
    !git clone https://github.com/alex-jk/datakit-smallholder-farmers-fall-2025.git

%cd {repo_dir}
%cd 'Prep Challenge- Translation'

**Load the farmers survey dataset**

In [ ]:
drive.mount('/content/drive')

export_input_to_parquet = False

!pip install duckdb -q

In [ ]:
import duckdb

csv_path = "/content/drive/MyDrive/DataKind Farmers Project/farmers_survey_dataset.csv"
parquet_path = "/content/drive/MyDrive/DataKind Farmers Project/farmers_survey_dataset.parquet"

con = duckdb.connect()

In [ ]:
if export_input_to_parquet:

  con.execute(f"""
      COPY (
          SELECT *
          FROM read_csv_auto('{csv_path}', sample_size=-1)
      )
      TO '{parquet_path}'
      (FORMAT PARQUET);
  """)

**Connect to the DB**

In [ ]:
parquet_path = "/content/drive/MyDrive/DataKind Farmers Project/farmers_survey_dataset.parquet"

con.execute(f"""
    CREATE OR REPLACE VIEW farmers AS
    SELECT *
    FROM read_parquet('{parquet_path}')
""")

In [ ]:
# Row count
row_count_df = con.execute("SELECT COUNT(*) AS n_rows FROM farmers").df()
n_rows = int(row_count_df.loc[0, "n_rows"])

# Column info
cols_info = con.execute("PRAGMA table_info('farmers')").df()
n_cols = cols_info.shape[0]
col_names = cols_info["name"].tolist()

print(f"DF row count: {n_rows}")
print(f"DF column count: {n_cols}")
print("Column names:")
print(col_names)

# Peek at data
con.execute("SELECT * FROM farmers LIMIT 5").df()

**Check unique question counts**

In [ ]:
questions_counts = con.execute("""
    SELECT
        question_id,
        question_topic,
        question_content,
        question_language,
        COUNT(*) AS n_responses
    FROM farmers
    GROUP BY
        question_id,
        question_topic,
        question_content,
        question_language
    ORDER BY
        n_responses DESC
""").df()

print(questions_counts.shape)
display(questions_counts)

**Check if each question ID corresponds to a unique question**

In [ ]:
# All question_ids where there is more than 1 distinct question_content
qid_conflicts = con.execute("""
    SELECT
        question_id,
        COUNT(DISTINCT question_content) AS n_question_contents
    FROM farmers
    GROUP BY question_id
    HAVING COUNT(DISTINCT question_content) > 1
    ORDER BY n_question_contents DESC
""").df()

qid_conflicts

Check english questions and responses

In [ ]:
pd.set_option("display.max_colwidth", None)

qid_counts_by_lang = con.execute("""
    SELECT
        question_language,
        COUNT(DISTINCT question_id) AS n_unique_question_ids
    FROM farmers
    GROUP BY question_language
    ORDER BY question_language
""").df()

qid_counts_by_lang["n_unique_question_ids_fmt"] = (
    qid_counts_by_lang["n_unique_question_ids"].astype(int).map("{:,}".format)
)

display(qid_counts_by_lang)

select_lang = "eng"  # or "swa", "lug", ...

sample_lang = con.execute("""
    SELECT
        question_id,
        question_topic,
        question_content,
        response_content,
        question_language
    FROM farmers
    WHERE question_language = ?
      AND question_content IS NOT NULL
      AND response_content IS NOT NULL
    ORDER BY random()
    LIMIT 20
""", [select_lang]).df()

print(f"\n-------Questions Sample {select_lang}------\n")
display(sample_lang)